# Tests du service owl generator

### Import ontoconnectlm package

In [1]:
import sys

sys.path.append("../")

 This notebook describes the test of the OWL Generator
- Objective: Generate a populated ontology file from extracted triples

### Test import

In [2]:
from ontoconnectlm.owl_generator import OwlGenerator

/home/maxence/other/.venv/lib/python3.12/site-packages/owlapy/static_funcs.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


### Input data (end-user)

In [3]:
# Exemples of triples

triples = [
    {'label': 'occuredAtTime',
  'head': "Détection d'oscillation de fréquence",
  'head_type': 'Event',
  'tail': '20/04/2012 à 07h11',
  'tail_type': 'Datetime'},

 {'label': 'occuredAtPlace',
  'head': "Détection d'oscillation de fréquence",
  'head_type': 'Event',
  'tail': 'Belgique',
  'tail_type': 'Location'},

 {'label': 'occuredAtTime',
  'head': "Coupure longue de l'ACR",
  'head_type': 'Event',
  'tail': '08/07/2003 à 08h40',
  'tail_type': 'Datetime'},

 {'label': 'occuredAtPlace',
  'head': "Coupure longue de l'ACR",
  'head_type': 'Event',
  'tail': 'Haute-Normandie',
  'tail_type': 'Location'},

 {'label': 'affectedComponent',
  'head': "Coupure longue de l'ACR",
  'head_type': 'Event',
  'tail': 'liaison 225 kV MASSY-PALAISEAU 4',
  'tail_type': 'PowerComponent'},

 {'label': 'occuredAtTime',
  'head': 'Coupure longue',
  'head_type': 'Event',
  'tail': '19/04/2016 à 16h32',
  'tail_type': 'Datetime'},

 {'label': 'occuredAtPlace',
  'head': 'Coupure longue',
  'head_type': 'Event',
  'tail': 'Bordeaux',
  'tail_type': 'Location'},

 {'label': 'affectedComponent',
  'head': 'Coupure longue',
  'head_type': 'Event',
  'tail': 'lignes 93 kV TONNERRE-NUITS-SAINT-GEORGES 1 et BORDEAUX-SAINT-JEAN 93 kV',
  'tail_type': 'PowerComponent'},

 {'label': 'impactedClient',
  'head': 'Coupure longue',
  'head_type': 'Event',
  'tail': 'SystemX',
  'tail_type': 'Client'},

 {'label': 'impactedClient',
  'head': 'Coupure longue',
  'head_type': 'Event',
  'tail': 'CEA',
  'tail_type': 'Client'},
  
 {'label': 'impactedClient',
  'head': 'Coupure du CNRS.',
  'head_type': 'Event',
  'tail': 'CNRS',
  'tail_type': 'Client'}
]

### Entity Linker definition

The entity linker is the dictionary that specifies the names of the desired classes in the ontology for the types of entities found in the triples. For example, if the triples mention entities of type "PowerComponent", this is where we specify that the associated class will be "Infrastructure".
Keys should be written in lowercase.

In [4]:
entity_linker = {
    "action" : "Action",

    "border" : "Border",

    "center_division" : "CenterDivision",

    "client" : "ThirdPartyActor",

    "criticality_level" : "CriticalityLevel",

    "datetime" : "Datetime",

    "duration" : "Duration",

    "event" : "Event",

    "event_impact" : "EventImpact",

    "external_event" : "OriginEvent",

    "location" : "GeographicalRegion",

    "powercomponent" : "PowerSystemComponent",

    "power" : "PowerFlow",

    "substation" : "Substation",

    "other" :   "Other"
}

### Definition of additional classes and relations

 Here, we specify the classes and properties to be included in the ontology.
By default, classes and properties are created from the triples. But we can also add them manually edit this method.


In [5]:
additional_classes = [
    "Event",
    "Action",
    "Coupling",
    "Disconnection",
    "Incident",
    "PowerOutage",
    "OriginEvent",
    "EventImpact",
    "Location",
    "Border",
    "GeographicalRegion",
    "CenterDivision",
    "ThirdPartyActor",
    "Datetime",
    "Duration",
    "PowerSystemComponent",
    "Transformer",
    "Line",
    "Generator",
    "Capacitor",
    "Substation",
    "Measurement",
    "PowerFlow",
    "Voltage",
    "CriticalityLevel"
]

additional_properties = [
    {"name" : "managed_by",
     "domain" : "PowerSystemComponent",
     "range" : "CenterDivision"},

    {"name" : "has_nominal_value",
     "domain" : "PowerSystemComponent",
     "range" : "Voltage"},

    {"name" : "located_in",
     "domain" : "PowerSystemComponent",
     "range" : "GeographicalRegion"},

    {"name" : "handled_by",
     "domain" : "Incident",
     "range" : "CenterDivision"},

    {"name" : "occured_on",
     "domain" : "Event",
     "range" : "PowerSystemComponent"},

    {"name" : "occured_in",
     "domain" : "Event",
     "range" : "Substation"},

    {"name" : "linked_to",
     "domain" : "Event",
     "range" : "Event"},

    {"name" : "consequence_of",
     "domain" : "Event",
     "range" : "Event"},

    {"name" : "similar_to",
     "domain" : "Event",
     "range" : "Event"},

    {"name" : "solved_by",
     "domain" : "Incident",
     "range" : "Action"},

    {"name" : "has_criticality",
     "domain" : "Incident",
     "range" : "CriticalityLevel"},

    {"name" : "has_impact",
     "domain" : "Incident",
     "range" : "EventImpact"},

    {"name" : "of_electrical_power",
     "domain" : "Event",
     "range" : "PowerFlow"},

    {"name" : "of_voltage",
     "domain" : "Event",
     "range" : "Voltage"},

    {"name" : "occured_at_datetime",
     "domain" : "Event",
     "range" : "Datetime"},

    {"name" : "lasted_for",
     "domain" : "Event",
     "range" : "Duration"},

    {"name" : "occured_in_place",
     "domain" : "Event",
     "range" : "GeographicalRegion"},

    {"name" : "occured_at_border",
     "domain" : "Event",
     "range" : "Border"},

    {"name" : "involved",
     "domain" : "Event",
     "range" : "ThirdPartyActor"},
]

### Creation of an owl genartor instance 

In [6]:
# Configurate the genartor 

config = {
    "entity_linker" : entity_linker,
    "additional_classes" : additional_classes,
    "additional_properties" : additional_properties
}

In [7]:
# Create a generator instance
owl_generator = OwlGenerator(
    triples = triples,
    config=config,
    onto_uri = "http://smd#"
)

## Genration of a populated ontology version

In [10]:
result_ontology = owl_generator.create_ontology()

print(result_ontology)

Saving /tmp/tmp_vujwkkg.owl..
<?xml version="1.0"?>
<rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
         xmlns:xsd="http://www.w3.org/2001/XMLSchema#"
         xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
         xmlns:owl="http://www.w3.org/2002/07/owl#"
         xml:base="http://smd"
         xmlns="http://smd#">

<owl:Ontology rdf:about="http://smd"/>

<owl:ObjectProperty rdf:about="#occuredattime"/>

<owl:ObjectProperty rdf:about="#occuredatplace"/>

<owl:ObjectProperty rdf:about="#affectedcomponent"/>

<owl:ObjectProperty rdf:about="#impactedclient"/>

<owl:ObjectProperty rdf:about="#managed_by">
  <rdfs:domain rdf:resource="#PowerSystemComponent"/>
  <rdfs:range rdf:resource="#CenterDivision"/>
</owl:ObjectProperty>

<owl:ObjectProperty rdf:about="#has_nominal_value">
  <rdfs:domain rdf:resource="#PowerSystemComponent"/>
  <rdfs:range rdf:resource="#Voltage"/>
</owl:ObjectProperty>

<owl:ObjectProperty rdf:about="#located_in">
  <rdfs:domain rdf:resour